In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/LowRankVortex.jl/")

  Activating environment at `/media/mat/HDD/LowRankVortex.jl/Project.toml`


In [2]:
using Revise
using LowRankVortex
using LinearAlgebra
using Statistics
using PotentialFlow
import PotentialFlow.Plates: Plate, Points, Blobs
import PotentialFlow.Motions: reset_velocity!
import PotentialFlow.Elements
import PotentialFlow.Properties: @property
using Interpolations
using Distributions
using Test

In [45]:
@testset "Test routine symmetric_analytical_jacobian_pressure with real freestream" begin
    atol = 1000*eps()

    Nv = 10
    Nx = 3*Nv

    x = rand(Nx)
    x0 = deepcopy(x)

    xsensors = collect(-2.0:0.5:10)
    sensors = complex(xsensors)
    Ny = length(sensors)

    U = randn() + 0.0*im
    freestream = Freestream(U);

    config = let Nv = Nv,
             U = U,
             ss = sensors, Δt = 1e-3, δ = 5e-2,
             ϵX = 1e-3, ϵΓ = 1e-4,
             β = 1.0,
             ϵY = 1e-16
            VortexConfig(Nv, U, ss, Δt, δ, ϵX, ϵΓ, β, ϵY)
    end

    sys = state_to_lagrange(x, config; isblob = true)
    @test typeof(sys[1][1])<:Vortex.Blob{Float64, Float64}

    sys = vcat(sys...)

    Jfull = analytical_jacobian_pressure(sensors, sys, freestream, 0.0)
    Jsymfull = symmetric_analytical_jacobian_pressure(xsensors, sys, freestream, 0.0)

    @test isapprox(Jfull, Jsymfull, atol = atol)

    # Test only on a subset of the vortices

    J = analytical_jacobian_pressure(sensors, sys, freestream, 1:Nv, 0.0)
    Jsym = symmetric_analytical_jacobian_pressure(xsensors, sys, freestream, 1:Nv, 0.0)

    @test isapprox(J[:,1:3*Nv], Jfull[:,1:3*Nv], atol = atol)
    @test isapprox(Jsym[:,1:3*Nv], Jfull[:,1:3*Nv], atol = atol)

    @test norm(J[:,3*Nv+1:end]) < atol
    @test norm(Jsym[:,3*Nv+1:end]) < atol
end

Test routine symmetric_analytical_jacobian_pressure with real freestream: Test Failed at In[45]:34
  Expression: isapprox(Jfull, Jsymfull, atol = atol)
   Evaluated: isapprox([-0.020228454352891324 -0.007320144512090477 … 0.00197139710740587 -0.021288790515288057; -0.030199460823862632 -0.011570524383773564 … 0.002298185870687682 -0.03067035848623708; … ; -0.0006218481276477345 -0.0011333723538668984 … 0.0005197590991284208 -0.0020249091567650676; -0.0005774987663901541 -0.001003887054412158 … 0.0004613650913501403 -0.001816220007420058], [-0.020219586149881976 -0.007325876005703322 … 0.0019733589609384414 -0.021283681609322282; -0.03017740419558086 -0.011580152228055692 … 0.002302182026930913 -0.03065914735681174; … ; -0.0006218776119599744 -0.001133453650782906 … 0.0005197801094441001 -0.0020248609348558974; -0.0005775211921539401 -0.0010039525637101916 … 0.00046138196778051487 -0.001816181324952718]; atol = 2.220446049250313e-13)
Stacktrace:
 [1] macro expansion
   @ In[45]:34 [inli

LoadError: [91mSome tests did not pass: 4 passed, 2 failed, 0 errored, 0 broken.[39m